# Data Cleaning 

#### 1. Import pandas library.

In [47]:
import pandas as pd

#### 2. Import pymysql and sqlalchemy as you have learnt in the lesson of importing/exporting data. 


In [48]:
import pymysql
import sqlalchemy 
from sqlalchemy import create_engine

#### 3. Create a mysql engine to set the connection to the server. Check the connection details in [this link](https://relational.fit.cvut.cz/dataset/Stats).

In [60]:
engine = create_engine('sqlite:///guest:relational@relational.fit.cvut.cz:3306/stats')

#### 4. Import the users table.

In [61]:
users = pd.read_sql_query("SELECT * FROM users", engine)

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: http://sqlalche.me/e/13/e3q8)

#### 5. Rename Id column to userId.

In [ ]:
user_id = "ALTER TABLE Id RENAME TO userId"
pd.read_sql_query(user_id, engine)

#### 6. Import the posts table. 

In [ ]:
posts = pd.read_sql_query("SELECT * FROM posts", engine)

#### 7. Rename Id column to postId and OwnerUserId to userId.

In [ ]:
post_id = "ALTER TABLE Id RENAME TO postId"
owner_user_id = "ALTER TABLE OwnerUserId RENAME TO userID"

#### 8. Define new dataframes for users and posts with the following selected columns:
**users columns**: userId, Reputation, Views, UpVotes, DownVotes  
**posts columns**: postId, Score, userID, ViewCount, CommentCount

#### 9. Merge the new dataframes you have created, of users and posts. 
You will need to make an inner [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

#### 10. How many missing values do you have in your merged dataframe? On which columns?

#### 11. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before going to the next step.

#### 12. Adjust the data types in order to avoid future issues. Which ones should be changed? 